# Purely data-driven model trained with both displacement and fmembran force fields

## Import packages

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

## User defined neural network
* A fully-connected feed-forward network
    * **n_input** - dimension of input, 2 in this case(x and y lcation)
    * **n_output** - dimension of output, 1 in this case (u - horizontal or v - vertical displacemnt)
    * **n_layer** - number of hidden layers
    * **n_nodes** - number of nodes of each hidden layer
* **two networks are defined seperately for u and v**

In [2]:
class Net(nn.Module):

    def __init__(self, n_input, n_output, n_layer, n_nodes):
        super(Net, self).__init__()
        self.n_layer = n_layer
        
        self.Input = nn.Linear(n_input, n_nodes)   # linear layer
        nn.init.xavier_uniform_(self.Input.weight) # wigths and bias initiation
        nn.init.normal_(self.Input.bias)

        self.Output = nn.Linear(n_nodes, n_output)
        nn.init.xavier_uniform_(self.Output.weight)
        nn.init.normal_(self.Output.bias)
        
        self.Hidden = nn.ModuleList() # hidden layer list
        for i in range(n_layer):
            self.Hidden.append(nn.Linear(n_nodes, n_nodes))
        for layer in self.Hidden:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.normal_(layer.bias)
        

    def forward(self, x):
        y = torch.tanh(self.Input(x)) # tanh activation function
        for layer in self.Hidden:
            y = torch.tanh(layer(y))
        y = self.Output(y)
        return y

In [3]:
def derivativs2(x, Net):
    
    w = Net(x)
    dw_x = []
    dw_y = []
    
    for i in range(w.size()[1]):
        
        #print(Net(x),func(x).view(-1,1),w)
        dw_xy = torch.autograd.grad(w[:,i], x, torch.ones_like(w[:,i]), retain_graph=True, 
                                    create_graph=True, allow_unused=True)
        dw_x.append(dw_xy[0][:,0].view(-1,1))
        dw_y.append(dw_xy[0][:,1].view(-1,1))

    return w, dw_x, dw_y

In [4]:
def loss_fcn(Net, X, Y):
     
    U_pred, dux, duy = derivativs2(X, Net)
    
    du_x, du_y = dux[0], duy[0]
    dv_x, dv_y = dux[1], duy[1]

    E, mu = 70, 0.3
    sig_x = (du_x + mu*dv_y)*E/(1 - mu**2) 
    sig_y = (dv_y + mu*du_x)*E/(1 - mu**2)
    sig_xy = (dv_x + du_y)*E/(1 + mu)/2.

    Y_pred = torch.cat((sig_x,sig_y,sig_xy), 1)
    
    loss = err(Y_pred, Y)
    
    return loss

## Preparing training data

In [5]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, X, Y):
        
        self.X = X
        self.Y = Y
        
    def __len__(self):
        
        return len(self.X)

    def __getitem__(self, index):
        
        x = self.X[index]
        y = self.Y[index]

        return x, y

In [6]:
def err(X, Y):
    
    return torch.mean(torch.mean((X-Y)**2))

In [7]:
data = pd.read_csv('rec-2.csv')
X_train = data.iloc[:, 5:7].to_numpy()
U = data.iloc[:,11:13].to_numpy()
LE = data.iloc[:,13:16].to_numpy()
sig = data.iloc[:,16:19].to_numpy()
U1 = U[:,0].reshape(-1, 1)
U2 = U[:,1].reshape(-1, 1)
eps11 = LE[:,0].reshape(-1, 1)
eps22 = LE[:,1].reshape(-1, 1)
eps12 = LE[:,2].reshape(-1, 1)
sig11 = sig[:,0].reshape(-1, 1)
sig22 = sig[:,1].reshape(-1, 1)
sig12 = sig[:,2].reshape(-1, 1)

In [8]:
X_train = X_train/10
X_train = torch.tensor(X_train, dtype=torch.float32, requires_grad=True)
U = torch.tensor(U, dtype=torch.float32)
U = U / 10
Sig = torch.tensor(sig, dtype=torch.float32)

In [9]:
# Prepare training data
Net_u = Net(2, 2, 5, 5)

In [10]:
# Construct neural network
# optimizer
nepoches = 3000
learning_rate = 1.0e-3
optimizer = torch.optim.Adam(Net_u.parameters(), lr=learning_rate)
training_set = Dataset(X_train, Sig)
training_generator = torch.utils.data.DataLoader(training_set, batch_size= 128)
training_loss = []
sig_loss = []
for epoch in range(nepoches):
    
    for X_batch, Y_batch  in training_generator:
        
        U_pred = Net_u(X_train)
        
        loss_uv = err(U_pred, U)
        
        loss_sig = loss_fcn(Net_u, X_batch, Y_batch)
        
        loss = loss_uv + loss_sig
        
        loss.backward()
         
        optimizer.step()
        optimizer.zero_grad()
        
    if (epoch+1) % 100 == 0:
        print(f'epoch:{epoch+1}: loss:{loss:.4e},loss1:{loss_uv:.4e},loss2:{loss_sig:.4e} ')
        
    if (epoch+1) % 10 == 0:
        
        U_pred = Net_u(X_train)
        
        loss = loss_fcn(Net_u, X_train, Sig)
        
        
        loss_uv = err(U_pred, U)
        
        training_loss.append(loss_uv)
        sig_loss.append(loss) 

epoch:100: loss:1.6472e-03,loss1:1.7540e-05,loss2:1.6297e-03 
epoch:200: loss:3.0868e-04,loss1:2.5144e-06,loss2:3.0616e-04 
epoch:300: loss:2.3217e-04,loss1:2.6986e-07,loss2:2.3190e-04 


KeyboardInterrupt: 

## Postprocessing

In [ ]:
torch.save({'Net_u':Net_u.state_dict()}, 'NN-Stress-5-5.pt')

In [ ]:
xx = [xx.detach().numpy() for xx in sig_loss]

In [ ]:
xx = [xx.detach().numpy() for xx in training_loss]

In [ ]:
pd.DataFrame(xx).to_clipboard()

In [ ]:
#X_train.requires_grad = True
U_pred = Net_u(X_train)
X = X_train[:,0].detach().numpy()*10
Y = X_train[:,1].detach().numpy()*10
U_pred = U_pred.detach().numpy()*10

u1 = U_pred[:,0].reshape(-1,1)
u2 = U_pred[:,1].reshape(-1,1)

In [ ]:
min(u1)

In [ ]:
np.corrcoef(U1.reshape(1,-1), u1.reshape(1,-1))

In [ ]:
np.corrcoef(U2.reshape(1,-1), u2.reshape(1,-1))

In [ ]:
# Then, "ALWAYS use sans-serif fonts"
plt.rcParams['font.family'] = 'Times New Roman'
fig, ax = plt.subplots(figsize=(5.8, 4.8)) 
surf = ax.scatter(X, Y, c = u1, vmin=0, vmax=0.14, cmap=cm.rainbow)
#cbar = fig.colorbar(ax)
cb = fig.colorbar(surf)
cb.ax.locator_params(nbins=7)
cb.ax.tick_params(labelsize=16)
#cb.set_label(label =r'$\sigma_xx (MPa)$', fontsize=16)
#cb.set_label(fontsize=16)
ax.axis('equal')
ax.set_xlabel('X Position (mm)', fontsize=18)
ax.set_ylabel('Y Position (mm)', fontsize=18)
for tick in ax.get_xticklabels():
    #tick.set_fontname('Times New Roman')
    tick.set_fontsize(16)
for tick in ax.get_yticklabels():
    #tick.set_fontname('Times New Roman')
    tick.set_fontsize(16)
#plt.savefig('Flat-U-NN10-20.png', dpi=600, transparent=True)
plt.show()

In [ ]:
_, dux, duy = derivativs2(X_train, Net_u)
du_x, du_y = dux[0], duy[0]
dv_x, dv_y = dux[1], duy[1]

E, mu = 70, 0.3
sig_x = (du_x + mu*dv_y)*E/(1 - mu**2) 
sig_y = (dv_y + mu*du_x)*E/(1 - mu**2)
sig_xy = (dv_x + du_y)*E/(1 + mu)/2.

sig_x = sig_x.detach().numpy()
sig_y = sig_y.detach().numpy() 
sig_xy = sig_xy.detach().numpy()

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'
fig, ax = plt.subplots(figsize=(5.8, 4.8)) 
surf = ax.scatter(X, Y, c = sig_x, vmin=0, vmax=1.0, cmap=cm.rainbow)
#cbar = fig.colorbar(ax)
cb = fig.colorbar(surf)
cb.ax.locator_params(nbins=7)
cb.ax.tick_params(labelsize=16)
#cb.set_label(label =r'$\sigma_xx (MPa)$', fontsize=16)
#cb.set_label(fontsize=16)
ax.axis('equal')
ax.set_xlabel('X Position (mm)', fontsize=18)
ax.set_ylabel('Y Position (mm)', fontsize=18)
for tick in ax.get_xticklabels():
    #tick.set_fontname('Times New Roman')
    tick.set_fontsize(16)
for tick in ax.get_yticklabels():
    #tick.set_fontname('Times New Roman')
    tick.set_fontsize(16)
#plt.savefig('Flat-S-NN5-5.png', dpi=600, transparent=True)
plt.show()

In [ ]:
np.corrcoef(sig22.reshape(1,-1), sig_y.reshape(1,-1))

In [ ]:
data_out = np.hstack([X.reshape(-1,1),Y.reshape(-1,1),u1,u2,sig_x,sig_y,sig_xy])
df_out = pd.DataFrame(data_out, columns=['X', 'Y', 'U', 'V', 'Sig_x', 'Sig_y', 'Sig_xy'])

In [ ]:
df_out.to_csv('Flat-Stress-NN-5-5.csv')